In [4]:
import pandas as pd
import plot
from defs import SlotConfig, NN, Database, PacketType
from plot import Plot
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

db = Database()

In [8]:
import re


idxs = [[], [], [], []]

rgx = r'df \d+ (\w+) no\-renew'

[ idxs[i].append((re.sub(rgx, r'\1', a[0]), a[1], i + 1)) for i, df in enumerate(dfs) for a in df.index ]

for i in range(4):
    dfs[i].index = pd.MultiIndex.from_tuples(idxs[i], names=['source', 'dns type', 'nn'])

df = pd.concat(dfs)

df = df.unstack(1).drop(columns=['TA', 'MA', 'PDay'], level=0)
# cols = []
# for m in ['TA', 'MA', 'PDay']:
#     for th in [1.0,1.5]:
#         for pt in ['nx, ']:
#     cols.append(())

# df)

# df = df.stack(1)
display(df)

with open('/Users/princio/Repo/princio/Paper-IEEEE-Access/_table.tex', 'w') as fp:
    t = df.to_latex()
    rgx = re.compile(r'\\cline\{\d-\d+\}', re.MULTILINE)
    t = rgx.sub('', t)
    t = re.sub(r'^\\multirow\[t\]\{\d\}\{\*\}\{(\w+)\}', r'\\multicolumn{6}{l}{\1} \\\\\n', t, 0, re.MULTILINE)
    t = re.sub(r'\\multicolumn\{(\d)\}\{\w\}', r'\\multicolumn{\1}{l}', t, 0, re.MULTILINE)
    t = t.replace(r'0.', '.')
    fp.write('''
\\begin{table*}
    \\centering
    \\resizebox*{\\textwidth}{!}{%\n''' + t + '''
    }
    \\caption{Bo ''' + str(1+i) + '''.}
    \\label{tab:results}
\\end{table*}
''')


UDay                     TAR                     MAR        ...  \
th           1.0         1.5         1.0         1.5         1.0        ...   
dns type      nx    ok    nx    ok    nx    ok    nx    ok    nx    ok  ...   
source  nn                                                              ...   
caphaw  1   0.01  0.85  0.03     1  0.77  0.12  0.66     0  0.23  0.88  ...   
        2   0.07  0.82  0.07     1  0.68  0.15  0.68     0  0.32  0.85  ...   
        3   0.04  0.86  0.04     1  0.80  0.13  0.80     0  0.20  0.87  ...   
        4   0.10  0.87  0.29     1  0.51  0.12  0.32     0  0.49  0.88  ...   
simda   1   0.05  0.15  0.08     1  0.91  0.12  0.86     0  0.09  0.88  ...   
        2   0.05  0.10  0.05  0.91  0.95  0.38  0.95  0.02  0.05  0.62  ...   
        3   0.03  0.14  0.03     1  0.96  0.21  0.96     0  0.04  0.79  ...   
        4   0.07  0.15  0.08     1  0.89  0.12  0.84     0  0.11  0.88  ...   
unknown 1   0.07  0.95  0.23     1  0.84  0.03  0.62     0  0.16  0.97  ...   
        2      0  0.32     0  0.48  0.78  0.66  0.78  0.31  0.22  0.34  ...   
        3   0.05  0.68  0.05     1  0.97  0.20  0.97     0  0.03  0.80  ...   
        4   0.07  0.95  0.23     1  0.84  0.03  0.62     0  0.16  0.97  ...   
zbot    1   0.13  0.10  0.66  0.99  0.40  0.13  0.11  0.00  0.60  0.87  ...   
        2   0.20  0.07  0.20  0.98  0.35  0.34  0.35  0.01  0.65  0.66  ...   
        3   0.07  0.10  0.07  0.98  0.54  0.16  0.54  0.01  0.46  0.84  ...   
        4   0.43  0.10  0.84  1.00  0.19  0.13  0.05  0.00  0.81  0.87  ...   

           first TA           A                   ADay                   
th              1.5         1.0         1.5        1.0        1.5        
dns type         nx    ok    nx    ok    nx    ok   nx    ok   nx    ok  
source  nn                                                               
caphaw  1      1.62     -  6100   273  6100   273  793   233  793   233  
        2      1.50     -  4924   136  4924   136  786   117  786   117  
        3      1.54     -  5204   192  5204   192  776   173  776   173  
        4      1.85     -  5244   153  5244   153  793   142  793   142  
simda   1      1.94     -   384  2058   384  2058   73   304   73   304  
        2      1.81  2.79   384  2058   384  2058   73   304   73   304  
        3      1.90     -   384  2058   384  2058   73   304   73   304  
        4      1.94     -   384  2058   384  2058   73   304   73   304  
unknown 1         1     -    64   192    64   192   43   101   43   101  
        2         1  1.53    64   192    64   192   43   101   43   101  
        3         1     -    64   192    64   192   43   101   43   101  
        4         1     -    64   193    64   193   43   102   43   102  
zbot    1      3.49  3.43  3832  7216  3832  7216  998  1002  998  1002  
        2      2.38  3.67  3832  7216  3832  7216  998  1002  998  1002  
        3      2.57  3.83  3832  7216  3832  7216  998  1002  998  1002  
        4      2.30     4  3832  7216  3832  7216  998  1002  998  1002  

[16 rows x 24 columns]

In [21]:
from http.client import MULTI_STATUS
import re
import dtale
import pandas as pd


df = pd.read_csv('overall_4.csv', index_col=[0,1], header=[0,1])

# df = df.unstack(0).sort_index(level=1, axis=0)

df.columns = pd.MultiIndex.from_tuples([tuple((col[0][0:-7], col[0][-3:], col[1][1:3]))for col in df.columns], names=['source', 'th', 'dns type'])

display(df)

source         df 46 zbot no-renew                                         \
th                             1.0                       1.5                
dns type                        ok           nx           ok           nx   
metric   how                                                                
A        sum           7216.000000  3832.000000  7216.000000  3832.000000   
ADay     sum           1002.000000   998.000000  1002.000000   998.000000   
MA       sum           6293.000000  3119.000000  7212.000000  3629.000000   
MAR      ratio            0.872090     0.813935     0.999446     0.947025   
PDay     sum            903.000000   573.000000     4.000000   158.000000   
TA       sum            923.000000   713.000000     4.000000   203.000000   
TAR      ratio            0.127910     0.186065     0.000554     0.052975   
TDR      ratio            0.525984     0.635264     0.996024     0.863322   
first MA mean             1.179000     2.754545     1.174651     2.194584   
first TA mean             3.987818     2.181501     4.000000     2.297468   

source         df 54 caphaw no-renew                                        \
th                               1.0                      1.5                
dns type                          ok           nx          ok           nx   
metric   how                                                                 
A        sum              153.000000  5244.000000  153.000000  5244.000000   
ADay     sum              142.000000   793.000000  142.000000   793.000000   
MA       sum              134.000000  2560.000000  153.000000  3550.000000   
MAR      ratio              0.875817     0.488177    1.000000     0.676964   
PDay     sum               19.000000   713.000000    0.000000   564.000000   
TA       sum               19.000000  2684.000000    0.000000  1694.000000   
TAR      ratio              0.124183     0.511823    0.000000     0.323036   
TDR      ratio              0.881988     0.526560    1.000000     0.584377   
first MA mean               4.432000     3.074803    4.373239     2.174522   
first TA mean               4.000000     1.671809         NaN     1.851064   

source         df 57 unknown no-renew                                    \
th                                1.0                    1.5              
dns type                           ok         nx          ok         nx   
metric   how                                                              
A        sum               193.000000  64.000000  193.000000  64.000000   
ADay     sum               102.000000  43.000000  102.000000  43.000000   
MA       sum               188.000000  10.000000  193.000000  24.000000   
MAR      ratio               0.974093   0.156250    1.000000   0.375000   
PDay     sum                 5.000000  40.000000    0.000000  33.000000   
TA       sum                 5.000000  54.000000    0.000000  40.000000   
TAR      ratio               0.025907   0.843750    0.000000   0.625000   
TDR      ratio               0.953271   0.518072    1.000000   0.565789   
first MA mean                1.068627   1.000000    1.068627   1.411765   
first TA mean                4.000000   1.175000         NaN   1.000000   

source         df 58 simda no-renew                                       
th                              1.0                      1.5              
dns type                         ok          nx           ok          nx  
metric   how                                                              
A        sum            2058.000000  384.000000  2058.000000  384.000000  
ADay     sum             304.000000   73.000000   304.000000   73.000000  
MA       sum            1801.000000   42.000000  2058.000000   62.000000  
MAR      ratio             0.875121    0.109375     1.000000    0.161458  
PDay     sum             257.000000   68.000000     0.000000   67.000000  
TA       sum             257.000000  342.000000     0.000000  322.000000  
TAR      ratio             

In [2]:
from http.client import MULTI_STATUS
import re
import dtale
import pandas as pd
import numpy as np

dfs = []
for i in range(4):
    df = pd.read_csv(f'overall_{i+1}.csv', index_col=[0,1], header=[0,1])

    df.columns = pd.MultiIndex.from_tuples([tuple((col[0][0:-7], col[0][-3:], col[1][1:3]))for col in df.columns], names=['source', 'th', 'dns type'])

    df = df.sort_index(level=[0,1,2], ascending=[True, True, False], axis=1)

    df = df[[col for col in df.columns if col[0].find('no-renew') >= 0]]

    s = 1 - df.loc[('PDay', 'sum')] / (df.loc[('ADay', 'sum')])
    s.name = ('UDay', 'ratio')

    df = pd.concat([df, s.to_frame().T])

    df = df.map(lambda x: '-' if np.isnan(x) else (f'{int(x)}' if x == int(x) else f'{round(x, 2):.2f}') )

    df = df.T.unstack(1)

    # df = df.drop(columns=['first MA', ], level=0)
    df = df.droplevel(1, axis=1)

    cols = [ (col, th) for col in ['UDay', 'TAR', 'MAR', 'first TA', 'TA', 'MA', 'A', 'PDay', 'ADay'] for th in ['1.0', '1.5']]

    df = df[cols]
    dfs.append(df)

with open('latex.txt', 'w') as fp:
    for i, df in enumerate(dfs):
        t = df.to_latex()
        rgx = re.compile(r'\\cline\{\d-\d+\}', re.MULTILINE)
        t = rgx.sub('', t)
        t = re.sub(r'\\multirow\[t\]\{2\}\{\*\}\{df \d+ (\w+) no-renew\}', r'\\multicolumn{6}{l}{\1} \\\\\n', t, re.MULTILINE)
        t = t.replace('\\multicolumn{2}{r}', '\\multicolumn{2}{l}')
        # t = t.replace(r'\cline{1-18}', '')
        fp.write('''
\\begin{table*}
    \\centering
    \\resizebox*{\\textwidth}{!}{%\n''' + t + '''
    }
    \\caption{Bo ''' + str(1+i) + '''.}
    \\label{tab:results}
\\end{table*}
''')

df

UDay         TAR         MAR       first TA  \
th                                1.0   1.5   1.0   1.5   1.0   1.5      1.0   
source                 dns type                                                
df 46 zbot no-renew    nx        0.43  0.84  0.19  0.05  0.81  0.95     2.18   
                       ok        0.10  1.00  0.13  0.00  0.87  1.00     3.99   
df 54 caphaw no-renew  nx        0.10  0.29  0.51  0.32  0.49  0.68     1.67   
                       ok        0.87     1  0.12     0  0.88     1        4   
df 57 unknown no-renew nx        0.07  0.23  0.84  0.62  0.16  0.38     1.18   
                       ok        0.95     1  0.03     0  0.97     1        4   
df 58 simda no-renew   nx        0.07  0.08  0.89  0.84  0.11  0.16     1.88   
                       ok        0.15     1  0.12     0  0.88     1        4   

                                         TA          MA           A        \
th                                1.5   1.0   1.5   1.0   1.5   1.0   1.5   
source                 dns type                                             
df 46 zbot no-renew    nx        2.30   713   203  3119  3629  3832  3832   
                       ok           4   923     4  6293  7212  7216  7216   
df 54 caphaw no-renew  nx        1.85  2684  1694  2560  3550  5244  5244   
                       ok           -    19     0   134   153   153   153   
df 57 unknown no-renew nx           1    54    40    10    24    64    64   
                       ok           -     5     0   188   193   193   193   
df 58 simda no-renew   nx        1.94   342   322    42    62   384   384   
                       ok           -   257     0  1801  2058  2058  2058   

                                PDay       ADay        
th                               1.0  1.5   1.0   1.5  
source                 dns type                        
df 46 zbot no-renew    nx        573  158   998   998  
                       ok        903    4  1002  1002  
df 54 caphaw no-renew  nx        713  564   793   793  
                       ok         19    0   142   142  
df 57 unknown no-renew nx         40   33    43    43  
                       ok          5    0   102   102  
df 58 simda no-renew   nx         68   67    73    73  
                       ok        257    0   304   304

In [124]:

df = pd.read_csv('overall.csv', index_col=[0,1,2], header=[0])
df = df.map(lambda x: '-' if np.isnan(x) else (f'{int(x)}' if x == int(x) else f'{round(x, 2):.2f}') )
df

df 46 zbot no-renew th_1.0 df 46 zbot no-renew th_1.5  \
nn       metric   how                                                           
gok_pos1 A        sum                         7216                       7216   
gnx_pos1 A        sum                         3832                       3832   
         TA       sum                         1521                        416   
         first TA mean                        2.83                       3.49   
         MA       sum                         2311                       3416   
         first MA mean                        3.12                       2.25   
         ADay     sum                          998                        998   
         PDay     sum                          865                        338   
gok_pos1 TA       sum                          928                         12   
         first TA mean                        3.98                       3.43   
         MA       sum                         6288                       7204   
         first MA mean                        1.19                       1.17   
         ADay     sum                         1002                       1002   
         PDay     sum                          903                          7   
         TAR      ratio                       0.13                       0.00   
         MAR      ratio                       0.87                       1.00   
         TDR      ratio                       0.53                       0.99   
gnx_pos1 TAR      ratio                       0.40                       0.11   
         MAR      ratio                       0.60                       0.89   
         TDR      ratio                       0.54                       0.75   

                        df 46 zbot renew th_1.0 df 46 zbot renew th_1.5  \
nn       metric   how                                                     
gok_pos1 A        sum                      8012                    8012   
gnx_pos1 A        sum                      4749                    4749   
         TA       sum                      1921                     659   
         first TA mean                     2.59                    3.36   
         MA       sum                      2828                    4090   
         first MA mean                     2.68                    1.92   
         ADay     sum                      1002                    1002   
         PDay     sum                       944                     519   
gok_pos1 TA       sum                      1029                      13   
         first TA mean                     3.98                    3.62   
         MA       sum                      6983                    7999   
         first MA mean                     1.01                       1   
         ADay     sum                      1002                    1002   
         PDay     sum                      1002                       8   
         TAR      ratio                    0.13                    0.00   
         MAR      ratio                    0.87                    1.00   
         TDR      ratio                    0.50                    0.99   
gnx_pos1 TAR      ratio                    0.40                    0.14   
         MAR      ratio                    0.60                    0.86   
         TDR      ratio                    0.51                    0.66   

                        df 54 caphaw no-renew th_1.0  \
nn       metric   how                                  
gok_pos1 A        sum                            273   
gnx_pos1 A        sum                           6100   
         TA       sum                           4698   
         first TA mean                          1.39   
         MA       sum                           1402   
         first MA mean                          3.08   
         ADay     sum                            793   
         PDay     sum                            788   
gok_pos1 TA       sum         

In [117]:

df

first MA       first TA         MAR        \
                                    mean           mean       ratio         
th                                   1.0   1.5      1.0   1.5   1.0   1.5   
source                 dns type                                             
df 46 zbot no-renew    nx           3.12  2.25     2.83  3.49  0.60  0.89   
                       ok           1.19  1.17     3.98  3.43  0.87  1.00   
df 54 caphaw no-renew  nx           3.08  2.88     1.39  1.62  0.23  0.34   
                       ok           4.42  4.36        4     -  0.88     1   
df 57 unknown no-renew nx              1  1.41     1.18     1  0.16  0.38   
                       ok           1.07  1.07        4     -  0.97     1   
df 58 simda no-renew   nx           5.12  4.12     1.90  1.94  0.09  0.14   
                       ok           1.20  1.19        4     -  0.88     1   

                                  TAR         TDR        ...  ADay        \
                                ratio       ratio        ...   sum         
th                                1.0   1.5   1.0   1.5  ...   1.0   1.5   
source                 dns type                          ...               
df 46 zbot no-renew    nx        0.40  0.11  0.54  0.75  ...   998   998   
                       ok        0.13  0.00  0.53  0.99  ...  1002  1002   
df 54 caphaw no-renew  nx        0.77  0.66  0.50  0.51  ...   793   793   
                       ok        0.12     0  0.87     1  ...   233   233   
df 57 unknown no-renew nx        0.84  0.62  0.52  0.57  ...    43    43   
                       ok        0.03     0  0.95     1  ...   101   101   
df 58 simda no-renew   nx        0.91  0.86  0.51  0.52  ...    73    73   
                       ok        0.12     0  0.54     1  ...   304   304   

                                   MA       PDay         TA        UDay        
                                  sum        sum        sum       ratio        
th                                1.0   1.5  1.0  1.5   1.0   1.5   1.0   1.5  
source                 dns type                                                
df 46 zbot no-renew    nx        2311  3416  865  338  1521   416  0.87  0.34  
                       ok        6288  7204  903    7   928    12  0.90  0.01  
df 54 caphaw no-renew  nx        1402  2062  788  770  4698  4038  0.99  0.97  
                       ok         239   273   34    0    34     0  0.15     0  
df 57 unknown no-renew nx          10    24   40   33    54    40  0.93  0.77  
                       ok         187   192    5    0     5     0  0.05     0  
df 58 simda no-renew   nx          34    55   69   67   350   329  0.95  0.92  
                       ok        1801  2058  257    0   257     0  0.85     0  

[8 rows x 22 columns]

In [41]:
df = pd.read_csv('overall.csv', index_col=[0,1,2])
# dtale.show(df, open_browser=True )
import math
import numpy as np
df.unstack(0)

df.map(lambda x: 'never' if np.isnan(x) else (f'{int(x)}' if x == int(x) else f'{round(x, 2)}') )


df 46 zbot no-renew th_1.0 df 46 zbot no-renew th_1.5  \
nn       metric   how                                                          
gok_pos1 A        sum                        7216                       7216   
gnx_pos1 A        sum                        3832                       3832   
         TA       sum                        1521                        416   
         first TA mean                       2.83                       3.49   
         MA       sum                        2311                       3416   
         first MA mean                       3.12                       2.25   
         ADay     sum                         998                        998   
         PDay     sum                         865                        338   
gok_pos1 TA       sum                         928                         12   
         first TA mean                       3.98                       3.43   
         MA       sum                        6288                       7204   
         first MA mean                       1.19                       1.17   
         ADay     sum                        1002                       1002   
         PDay     sum                         903                          7   
         TAR      sum                        0.13                        0.0   
         MAR      sum                        0.87                        1.0   
         TDR      sum                        0.53                       0.99   
gnx_pos1 TAR      sum                         0.4                       0.11   
         MAR      sum                         0.6                       0.89   
         TDR      sum                        0.54                       0.75   

                       df 46 zbot renew th_1.0 df 46 zbot renew th_1.5  \
nn       metric   how                                                    
gok_pos1 A        sum                     8012                    8012   
gnx_pos1 A        sum                     4749                    4749   
         TA       sum                     1921                     659   
         first TA mean                    2.59                    3.36   
         MA       sum                     2828                    4090   
         first MA mean                    2.68                    1.92   
         ADay     sum                     1002                    1002   
         PDay     sum                      944                     519   
gok_pos1 TA       sum                     1029                      13   
         first TA mean                    3.98                    3.62   
         MA       sum                     6983                    7999   
         first MA mean                    1.01                       1   
         ADay     sum                     1002                    1002   
         PDay     sum                     1002                       8   
         TAR      sum                     0.13                     0.0   
         MAR      sum                     0.87                     1.0   
         TDR      sum                      0.5                    0.99   
gnx_pos1 TAR      sum                      0.4                    0.14   
         MAR      sum                      0.6                    0.86   
         TDR      sum                     0.51                    0.66   

                       df 54 caphaw no-renew th_1.0  \
nn       metric   how                                 
gok_pos1 A        sum                           273   
gnx_pos1 A        sum                          6100   
         TA       sum                          4698   
         first TA mean                         1.39   
         MA       sum                          1402   
         first MA mean                         3.08   
         ADay     sum                           793   
         PDay     sum                           788   
gok_pos1 TA       sum                            34   
         first TA mean      

In [15]:
df = pd.read_csv('df 54 caphaw no-renew.csv', index_col=None)
dtale.show(df, open_browser=True)

In [4]:
config = SlotConfig(
    sps=3600,
    onlyfirsts='global',
    packet_type=PacketType.Q,
    nn=NN.NONE,
    th=0.5,
    wl_th=0,
    wl_col='dn',
    range=(0,8),
    pcap_id=None
)

response = {
    "message": "Received data",
    "person": {
        "sps": config.sps,
        "nn": config.nn,
    }
}
plot = Plot(db, config)

a = plot.byfp()

a[2].show()
plt.show()

In [2]:
pcaps = pd.read_sql("SELECT * FROM PCAPMW WHERE DGA=2", db.engine)[["pcap_id","pcap_name","qr","q","u","duration","dga"]]

pcaps

,pcap_id,pcap_name,qr,q,u,duration,dga
0,45,25-2_2014-01-25_capture_win3.pcap,471881,235966,2003,147.61,2
1,54,69_2014-04-07_capture-win17.pcap,63838,33032,5746,1005.10,2
2,56,6_2013-08-20_capture-win6.pcap,395548,200163,3618,917.45,2
3,57,71_2014-04-07_capture-win19.fixed.pcap,17216,9092,2027,895.32,2
4,52,355-1_2018-05-07_capture-win6.pcap,7487792,4083349,1005,163.25,2
5,51,353-1_353-1.5001-10000.pcap,1686,857,783,0.02,2
6,42,225-1_capture_win1.fixed.pcap,421482,211281,1002,134.95,2
7,50,353-1_353-1.0-5000.pcap,1261,631,452,0.03,2
8,55,6_2013-08-20_capture-win11.pcap,381249,190728,2530,306.07,2
9,46,25-6_2014-12-20_capture-win3.pcap,13622,6819,4302,1010.27,2


In [1]:
import pandas as pd
import plot
from defs import SlotConfig, NN, Database, PacketType
from plot import Plot
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Qt5Agg')

db = Database()

pcaps = pd.read_sql("SELECT * FROM PCAPMW WHERE DGA=2", db.engine)[["pcap_id","pcap_name","qr","q","u","duration","dga"]]

for _,pcap in pcaps.iterrows():
    config = SlotConfig(
        sps=3600 * 24,
        onlyfirsts='all',
        packet_type=PacketType.R,
        nn=NN.NONE,
        th=0.5,
        wl_th=0,
        wl_col='dn',
        range=(0,8),
        pcap_id=pcap["pcap_id"]
    )
    
    plot = Plot(db, config)
    
    a = plot.by_r()
    
    plt.show()

KeyboardInterrupt: 